In [2]:
from tqdm import tqdm

from openff.recharge.conformers import ConformerGenerator, ConformerSettings
from openff.recharge.esp import ESPSettings
#from openff.recharge.esp.psi4 import Psi4ESPGenerator
from openff_psi4_debug import Psi4ESPGenerator
from openff.recharge.esp.storage import MoleculeESPRecord, MoleculeESPStore
from openff.recharge.grids import LatticeGridSettings
from openff.recharge.utilities.molecule import smiles_to_molecule
from openff.units.elements import SYMBOLS
from rdkit.Chem import AllChem


import os
import numpy as np
import random

In [3]:
#load our molecule to create the conformers

molecule = smiles_to_molecule("OCC(O)CO")
rdmol = molecule.to_rdkit()


In [4]:
AllChem.EmbedMultipleConfs(rdmol, numConfs=10, randomSeed=42)

In [5]:
from rdkit.Chem.rdForceFieldHelpers import MMFFOptimizeMolecule

In [6]:
#test the optimizer

optimize_MMFF = MMFFOptimizeMolecule(rdmol,mmffVariant = 'MMFF94',confId=3) #,confId=1,mmffvariant='MMFF94'
optimize_MMFF



0

In [7]:
#test all the conformers

for id in range(rdmol.GetNumConformers()):
    optimize_MMFF = MMFFOptimizeMolecule(rdmol,mmffVariant = 'MMFF94',confId=3) #,confId=1,mmffvariant='MMFF94'
    print(f'conformer:{id}, optimize (0 = converge, 1 = more iterations req): {optimize_MMFF}')

conformer:0, optimize (0 = converge, 1 = more iterations req): 0
conformer:1, optimize (0 = converge, 1 = more iterations req): 0
conformer:2, optimize (0 = converge, 1 = more iterations req): 0
conformer:3, optimize (0 = converge, 1 = more iterations req): 0
conformer:4, optimize (0 = converge, 1 = more iterations req): 0
conformer:5, optimize (0 = converge, 1 = more iterations req): 0
conformer:6, optimize (0 = converge, 1 = more iterations req): 0
conformer:7, optimize (0 = converge, 1 = more iterations req): 0
conformer:8, optimize (0 = converge, 1 = more iterations req): 0
conformer:9, optimize (0 = converge, 1 = more iterations req): 0


In [8]:
rdmol.GetConformer(1)

In [9]:
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.ipython_3d = True
import py3Dmol


In [10]:
#do the conformers look structurally sensible

for id in range(rdmol.GetNumConformers()):
    IPythonConsole.drawMol3D(rdmol,confId=id)


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [11]:
optimize_MMFF

0

In [12]:
#now create a series of xyz files so we can run the conformers with ASE, first convert our conformers to xyz format and then write to file

from openff.units import unit

cwd = os.getcwd()

conformers = []



for confs in range(rdmol.GetNumConformers()):
    conformer = np.zeros((rdmol.GetConformer(confs).GetNumAtoms(), 3))
    for atom_index, coordinates in enumerate(rdmol.GetConformer(confs).GetPositions()):
        conformer[atom_index, :] = coordinates
    conformers.append(conformer * unit.angstrom)


for number ,conformer in enumerate(conformers):
    atoms = [
            {
                "element": SYMBOLS[atom.atomic_number],
                "x": conformer[index, 0],
                "y": conformer[index, 1],
                "z": conformer[index, 2],
            }
            for index, atom in enumerate(molecule.atoms)
        ]
    xyz = f'{molecule.n_atoms}\n{molecule.to_smiles()}\n'
    for row in atoms:
        xyz += f"{row['element']}\t{np.around(row['x'].magnitude,decimals=6)}\t{np.around(row['y'].magnitude,decimals=6)}\t{np.around(row['z'].magnitude, decimals=6)}\n"
    try:
        f = open(cwd+f"/conformer_{number}.xyz", 'x')
        f.write(xyz)
        f.close()
    except FileExistsError:
        continue

FileExistsError: [Errno 17] File exists: '/mnt/storage/nobackup/nca121/test_jobs/QM_ESP_Psi4/QM_ESP_Psi4/conformer_0.xyz'

In [18]:
from ase.calculators.gaussian import Gaussian, GaussianOptimizer
import ase.io
from ase.visualize import view
!module load gaussian/09




In [19]:
Gaussian.command = '/mnt/storage/apps/eb/software/gaussian/09/g09 < PREFIX.com > PREFIX.log'

In [20]:
test_atoms = ase.io.read('conformer_0.xyz')

In [21]:
view(test_atoms)

<Popen: returncode: None args: ['/mnt/nfs/home/nca121/mambaforge/envs/openff...>

In [22]:
calc_opt = Gaussian(label = 'conformer_0_opt',
                    method = 'hf',
                    basis = '6-31G*',
                    scf='qc'
                    )
opt = GaussianOptimizer(test_atoms, calc_opt)
opt.run(fmax='tight', steps=100)



No executable for file l1.exe.

Search path GAUSS_EXEDIR is ""
: No such file or directory
/bin/sh: line 1:  2261 Segmentation fault      /mnt/storage/apps/eb/software/gaussian/09/g09 < conformer_0_opt.com > conformer_0_opt.log


CalculationFailed: Calculator "gaussian" failed with command "/mnt/storage/apps/eb/software/gaussian/09/g09 < conformer_0_opt.com > conformer_0_opt.log" failed in /mnt/storage/nobackup/nca121/test_jobs/QM_ESP_Psi4/QM_ESP_Psi4 with error code 139